In [1]:
#主に特徴量エンジニアリングとデータの分割の順番を変更し、評価指標としてaccuracy_scoreからf1_scoreを使用するように変更しています。また、アンサンブルについても加重平均化を使用するように変更しています。

In [3]:
#ライブラリのインポート
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix

#データの読み込み
train = pd.read_csv("./original/train.csv")
test = pd.read_csv("./original/test.csv")

#特徴量の追加
train["CPU_performance"] = train["battery_power"] * train["clock_speed"]
train["screen_area"] = train["sc_h"] * train["sc_w"]
train["total_camera_pixels"] = train["pc"] + train["fc"] + train["m_dep"]
train["total_features"] = train["ram"] + train["sc_h"] + train["sc_w"] + train["battery_power"]

test["CPU_performance"] = test["battery_power"] * test["clock_speed"]
test["screen_area"] = test["sc_h"] * test["sc_w"]
test["total_camera_pixels"] = test["pc"] + test["fc"] + test["m_dep"]
test["total_features"] = test["ram"] + test["sc_h"] + test["sc_w"] + test["battery_power"]

#特徴量と目的変数に分ける
X = train.drop("price_range", axis=1)
y = train["price_range"]

#特徴量のスケーリング
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(test)

#データを学習用と評価用に分割
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=1)

#モデルの作成と学習
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=1)
xgb_model.fit(X_train, y_train)

#評価用データの価格帯の予測
y_pred_rf = rf_model.predict(X_eval)
y_pred_xgb = xgb_model.predict(X_eval)

#アンサンブルした予測結果を取得
y_pred = np.round((y_pred_rf * 0.7 + y_pred_xgb * 0.3)).astype(int)

#f1_scoreによる評価
f1 = f1_score(y_eval, y_pred, average="weighted")
print("f1_score:", f1)

#予測結果の出力
y_pred = np.round((rf_model.predict(X_test) * 0.7 + xgb_model.predict(X_test) * 0.3)).astype(int)
output = pd.DataFrame({"id": test["id"], "price_range": y_pred})
output.to_csv("./submission/submission_rf-xgb_v4.csv", index=False, header=False)

f1_score: 0.38319179134601616


In [4]:
output

,id,price_range
0,1,2
1,2,0
2,6,3
3,10,1
4,12,1
...,...,...
795,1978,3
796,1980,1
797,1982,3
798,1988,2
